In [2]:
import pandas as pd

# Load the datasets
applications_df = pd.read_csv('cleaned_applications.csv')
branches_df = pd.read_csv('cleaned_branches.csv')
customers_df = pd.read_csv('cleaned_customers.csv')
defaults_df = pd.read_csv('cleaned_defaults.csv')
loans_df = pd.read_csv('cleaned_loans.csv')
transactions_df = pd.read_csv('cleaned_transactions.csv')

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [3]:
# Merge loans_df and defaults_df DataFrames using the 'Loan_ID' column, keeping only the loans that appear in both dataframes.
merged_defaults_df = pd.merge(loans_df, defaults_df, on='Loan_ID', how='inner')


In [4]:
# Convert the date columns in the merged DataFrame to datetime objects using the correct column name.
merged_defaults_df['Disbursal_Date'] = pd.to_datetime(merged_defaults_df['Disbursal_Date'])
merged_defaults_df['Default_Date'] = pd.to_datetime(merged_defaults_df['Default_Date'])

# Calculate the time difference between 'Default_Date' and 'Disbursal_Date'
merged_defaults_df['Time_to_Default'] = (merged_defaults_df['Default_Date'] - merged_defaults_df['Disbursal_Date']).dt.days

# Display the first few rows of the merged DataFrame with the new column
display(merged_defaults_df.head())

Loan_ID Customer_ID_x  Loan_Amount  Interest_Rate  Loan_Term  \
0  L000009       C028962      3104013          11.59         12   
1  L000019       C055991       678843           9.34         36   
2  L000025       C037085      4331567          10.14         12   
3  L000039       C011104      4291400          12.28         12   
4  L000050       C057771       626523          11.93         60   

  Disbursal_Date Repayment_Start_Date Repayment_End_Date Loan_Status  \
0     2023-05-24           2023-06-24         2024-05-24     Overdue   
1     2020-06-09           2020-07-09         2023-06-09     Overdue   
2     2020-08-25           2020-09-25         2021-08-25      Closed   
3     2022-02-13           2022-03-13         2023-02-13      Closed   
4     2024-11-20           2024-12-20         2029-11-20      Active   

   Overdue_Amount  ...  Collateral_Details Default_ID Customer_ID_y  \
0               0  ...      Not Applicable    D004096       C000009   
1           24132  ...      Not Applicable    D008198       C000019   
2           36601  ...      Not Applicable    D006597       C000025   
3               0  ...      Not Applicable    D000898       C000039   
4               0  ...             Vehicle    D005115       C000050   

  Default_Date Default_Amount     Default_Reason Recovery_Status  \
0   2022-09-05          48655   Business Failure         Partial   
1   2023-04-09          47609              Other         Partial   
2   2022-03-31          23052  Medical Emergency         Partial   
3   2021-04-21          33041   Business Failure         Partial   
4   2023-11-14          28774   Business Failure         Partial   

  Recovery_Amount  Legal_Action Time_to_Default  
0           45016            No            -261  
1               0           Yes            1034  
2           45893           Yes             583  
3            4066            No            -298  
4           23114            No            -372  

[5 rows x 21 columns]

In [5]:
# Calculate the average time to default
average_time_to_default = merged_defaults_df['Time_to_Default'].mean()

# Print the average time to default
print(f"The average time to default for defaulted loans is: {average_time_to_default:.2f} days")

The average time to default for defaulted loans is: 206.67 days


In [6]:
# Merge merged_defaults_df with applications_df to get the Loan_Purpose
merged_defaults_with_purpose_df = pd.merge(merged_defaults_df, applications_df[['Loan_ID', 'Loan_Purpose']], on='Loan_ID', how='left')

# Group by 'Loan_Purpose' and calculate the mean of 'Time_to_Default'
average_time_by_purpose = merged_defaults_with_purpose_df.groupby('Loan_Purpose')['Time_to_Default'].mean()

# Sort the results in ascending order
sorted_average_time_by_purpose = average_time_by_purpose.sort_values(ascending=True)

# Display the sorted results
display(sorted_average_time_by_purpose)

Loan_Purpose
Education          185.538296
Business           196.201754
Home Renovation    197.039660
Vehicle            227.274455
Personal           230.888418
Name: Time_to_Default, dtype: float64

In [7]:
# Merge merged_defaults_with_purpose_df with customers_df including available demographic columns using the correct Customer ID column
merged_defaults_customers_df = pd.merge(
    merged_defaults_with_purpose_df,
    customers_df[['Customer_ID', 'Gender', 'Marital_Status', 'Employment_Status']],
    left_on='Customer_ID_x',
    right_on='Customer_ID',
    how='left'
)

# Group by 'Gender' and calculate the mean of 'Time_to_Default'
average_time_by_gender = merged_defaults_customers_df.groupby('Gender')['Time_to_Default'].mean()

# Group by 'Marital_Status' and calculate the mean of 'Time_to_Default'
average_time_by_marital_status = merged_defaults_customers_df.groupby('Marital_Status')['Time_to_Default'].mean()

# Group by 'Employment_Status' and calculate the mean of 'Time_to_Default'
average_time_by_employment_status = merged_defaults_customers_df.groupby('Employment_Status')['Time_to_Default'].mean()

# Display the calculated average time to default for each demographic category
print("Average time to default by Gender:")
display(average_time_by_gender)

print("\nAverage time to default by Marital Status:")
display(average_time_by_marital_status)

print("\nAverage time to default by Employment Status:")
display(average_time_by_employment_status)

Average time to default by Gender:


Gender
Female    195.178419
Male      197.471414
Other     227.113531
Name: Time_to_Default, dtype: float64


Average time to default by Marital Status:


Marital_Status
Divorced    212.388997
Married     199.728904
Single      207.821796
Name: Time_to_Default, dtype: float64


Average time to default by Employment Status:


Employment_Status
Salaried         215.773970
Self-Employed    193.380806
Unemployed       210.396444
Name: Time_to_Default, dtype: float64